In [1]:
from room import Room
from person import Person
from action import Action
import metacode_helpers

In [2]:
import pandas as pd
import numpy as np

In [3]:
# initialise all actions
ALL_POSSIBLE_ACTIONS = metacode_helpers.initialise_all_actions()
ALL_CONVERSATION_ACTIONS = [action for action in ALL_POSSIBLE_ACTIONS if action.get_action_type() == "conversation"]
ALL_ROOM_ACTIONS = [action for action in ALL_POSSIBLE_ACTIONS if action.get_action_type() == "room"] 

In [4]:
# Create some rooms
HALL = Room("Hall")
KITCHEN = Room("Kitchen")
LOUNGE = Room("Lounge")

# Add adjacencies to the rooms (HALL is adjacent to KITCHEN and LOUNGE)
HALL.add_adjacent_room(KITCHEN)
HALL.add_adjacent_room(LOUNGE)

# Assign the Hall to be the entry to the house
HOUSE_ENTRY_ROOM = HALL 

ROOMS = [HALL, KITCHEN, LOUNGE]

In [5]:
names = metacode_helpers.get_names(num_names=32)
np.random.seed(42)
ALL_PEOPLE = []
for personality_number in range(7):
    personality_vector = np.random.uniform(low = -1, high = 1, size = 5)
    initial_emotional_state_vector = np.random.uniform(low = -1, high = 1, size = 3)
    initial_location_state = np.random.choice(ROOMS)
    new_person = Person(name = names[personality_number], location_state=initial_location_state, all_possible_actions=ALL_POSSIBLE_ACTIONS, personality_vector=personality_vector, emotional_state_vector=initial_emotional_state_vector, conversation_partner=None)
    ALL_PEOPLE.append(new_person)

In [6]:
ALL_PEOPLE

[Robin, Patrick, Ronnie, Jacquelyn, Phyllis, Agnes, Dorothy]

In [7]:
ALL_POSSIBLE_ACTIONS

array([shouts, shares, leaves_conversation, makes_smalltalk,
       makes_deeptalk, gives_controversial_opinion, changes_the_topic,
       shows_interest, shows_concern, laughs, jokes, gives_an_insult,
       brags, yawns, supports, rants, stands_in_corner, leaves_room,
       starts_conversation], dtype=object)

In [8]:
from collections import defaultdict

action_log = defaultdict(list)
person_before_action_log = defaultdict(list)
person_after_action_log = defaultdict(list)
partner_before_action_log = defaultdict(list)
partner_after_action_log = defaultdict(list)
room_log = defaultdict(list)

def log_snapshot(log, snapshot, turn_number, round_number):
    log["turn_number"].append(turn_number)
    log["round_number"].append(round_number)
    for key, val in snapshot.items():
        log[key].append(val)

turn_number = 0
for round_number in range(14):
    for person in ALL_PEOPLE:
        partner = person.get_conversation_partner()
        log_snapshot(person_before_action_log, person.get_snapshot(), turn_number, round_number)
        if person.has_conversation_partner():
            log_snapshot(partner_before_action_log, person.get_conversation_partner().get_snapshot(), turn_number, round_number)

        # The person can start a conversation only if there's someone free to chat in their current room.
        available_room_act = ALL_ROOM_ACTIONS if person.get_location_state().is_someone_free_to_chat(person) else [action for action in ALL_ROOM_ACTIONS if action.get_name() != "starts_conversation"]
        #print(available_room_act, "free?", person.get_location_state().is_someone_free_to_chat(person))
        # If in a conversation, the person can select any action, including leaving.
        action = person.action_selection(available_conv_act=ALL_CONVERSATION_ACTIONS) if person.has_conversation_partner() else person.action_selection(available_room_act=available_room_act)
        
        # If in a conversation, update the person & conversation partner's emotional state according to the action
        if person.has_conversation_partner():
            # Change this person's emotional state 
            person.update_emotional_state_vector(action, isOwnAction = True)
            # Change the partner's emotional state 
            partner.update_emotional_state_vector(action, isOwnAction = False)

            if action.get_name() == "leaves_conversation":
                # The conversation has ended, so both people now don't have conversation partners.
                person.set_conversation_partner(None)
                partner.set_conversation_partner(None)
                
        else: # Otherwise, deal with the room action
            if action.get_name() == "leaves_room":
                next_room = person.get_next_room()
                person.move_to_room(next_room)          
            if action.get_name() == "starts_conversation":
                # NOTE whether this is possible is handled by the action selection function
                # Find a person in the room who is free to chat
                current_room = person.get_location_state()
                partner = current_room.get_new_conversation_partner_for(person)
                person.set_conversation_partner(partner)
                partner.set_conversation_partner(person)
        
        log_snapshot(action_log, action.get_snapshot(), turn_number, round_number)    
        log_snapshot(person_after_action_log, person.get_snapshot(), turn_number, round_number)
        if partner:
            log_snapshot(partner_after_action_log, partner.get_snapshot(), turn_number, round_number)

        message = person.get_name(), ' '.join(action.get_name().split("_"))
        #print(round_number)
        #print(turn_number)
        #if partner:
            #print(message, 'To:', partner.get_name())
        #else:
            #print(message)
        turn_number += 1

#pd.DataFrame(action_log)
pd.DataFrame(person_before_action_log)


#print(person_after_action_log)
##print(partner_before_action_log)
#print(partner_after_action_log)
#print(room_log)#

,turn_number,round_number,name,personality_vector,emotional_state,base_action_probs,location_state,conversation_partner
0,0,0,Robin,"[-0.250919762305275, 0.9014286128198323, 0.463...","[-0.6880109593275947, -0.8838327756636011, 0.7...",{ =========================== ...,Lounge,None
1,1,0,Patrick,"[-0.9588310114083951, 0.9398197043239886, 0.66...","[-0.6331909802931324, -0.39151551408092455, 0....",{ =========================== ...,Hall,None
2,2,0,Ronnie,"[-0.4175417196039162, 0.22370578944475894, -0....","[-0.08786003156592814, 0.5703519227860272, -0....",{ =========================== ...,Lounge,None
3,3,0,Jacquelyn,"[0.9664617716135764, -0.06647421350404015, 0.7...","[-0.9734700776802669, 0.8844035113697055, 0.12...",{ =========================== ...,Kitchen,None
4,4,0,Phyllis,"[-0.39077246165325863, -0.8046557719872323, 0....","[-0.00964617977745963, -0.9312229577695632, 0....",{ =========================== ...,Kitchen,None
...,...,...,...,...,...,...,...,...
93,93,13,Ronnie,"[-0.4175417196039162, 0.22370578944475894, -0....","[-0.08786003156592814, 0.5703519227860272, -0....",{ =========================== ...,Kitchen,None
94,94,13,Jacquelyn,"[0.9664617716135764, -0.06647421350404015, 0.7...","[-0.8624700776802668, 1.0, 0.5080764356910785]",{ =========================== ...,Lounge,Patrick
95,95,13,Phyllis,"[-0.39077246165325863, -0.8046557719872323, 0....","[0.12735382022254038, -0.5102229577695628, 0.9...",{ =========================== ...,Kitchen,Agnes
96,96,13,Agnes,"[0.32504456870796394, -0.3765778478211781, 0.0...","[0.985, 0.8312656467222294, 0.9701]",{ =========================== ...,Kitchen,Phyllis


In [9]:
pd.DataFrame(person_after_action_log)

,turn_number,round_number,name,personality_vector,emotional_state,base_action_probs,location_state,conversation_partner
0,0,0,Robin,"[-0.250919762305275, 0.9014286128198323, 0.463...","[-0.6880109593275947, -0.8838327756636011, 0.7...",{ =========================== ...,Lounge,Dorothy
1,1,0,Patrick,"[-0.9588310114083951, 0.9398197043239886, 0.66...","[-0.6331909802931324, -0.39151551408092455, 0....",{ =========================== ...,Kitchen,None
2,2,0,Ronnie,"[-0.4175417196039162, 0.22370578944475894, -0....","[-0.08786003156592814, 0.5703519227860272, -0....",{ =========================== ...,Lounge,None
3,3,0,Jacquelyn,"[0.9664617716135764, -0.06647421350404015, 0.7...","[-0.9734700776802669, 0.8844035113697055, 0.12...",{ =========================== ...,Kitchen,None
4,4,0,Phyllis,"[-0.39077246165325863, -0.8046557719872323, 0....","[-0.00964617977745963, -0.9312229577695632, 0....",{ =========================== ...,Kitchen,Agnes
...,...,...,...,...,...,...,...,...
93,93,13,Ronnie,"[-0.4175417196039162, 0.22370578944475894, -0....","[-0.08786003156592814, 0.5703519227860272, -0....",{ =========================== ...,Kitchen,None
94,94,13,Jacquelyn,"[0.9664617716135764, -0.06647421350404015, 0.7...","[-0.8604700776802668, 1.0, 0.5120764356910785]",{ =========================== ...,Lounge,Patrick
95,95,13,Phyllis,"[-0.39077246165325863, -0.8046557719872323, 0....","[0.1373538202225404, -0.46022295776956285, 0.9...",{ =========================== ...,Kitchen,Agnes
96,96,13,Agnes,"[0.32504456870796394, -0.3765778478211781, 0.0...","[0.99, 0.8332656467222294, 0.9711]",{ =========================== ...,Kitchen,Phyllis


In [10]:
#Write to CSV files
pd.DataFrame(action_log).to_csv('action.csv')
pd.DataFrame(person_before_action_log).to_csv('personbefore.csv')
pd.DataFrame(person_after_action_log).to_csv('personafter.csv')
pd.DataFrame(partner_before_action_log).to_csv('partnerbefore.csv')
pd.DataFrame(partner_after_action_log).to_csv('partnerafter.csv')
pd.DataFrame(room_log).to_csv('room.csv')


In [47]:
#merging all the csv files into one big Dataframe
person_before_action_log = pd.DataFrame(person_before_action_log)
partner_before_action_log = pd.DataFrame(partner_before_action_log)
action_log = pd.DataFrame(action_log)
temp1 = pd.merge(person_before_action_log,partner_before_action_log, on= 'turn_number',how='outer')
temp2 = pd.merge(temp1,action_log, on= 'turn_number',how='outer')

pd.DataFrame(temp2).to_csv('combined1.csv')

    turn_number  round_number                 name
0             0             0  starts_conversation
1             1             0          leaves_room
2             2             0     stands_in_corner
3             3             0     stands_in_corner
4             4             0  starts_conversation
..          ...           ...                  ...
93           93            13     stands_in_corner
94           94            13                rants
95           95            13    changes_the_topic
96           96            13                brags
97           97            13                brags

[98 rows x 3 columns]
    turn_number  round_number       name  \
0             0             0      Robin   
1             1             0    Patrick   
2             2             0     Ronnie   
3             3             0  Jacquelyn   
4             4             0    Phyllis   
..          ...           ...        ...   
93           93            13     Ronnie   
94           

    Unnamed: 0  turn_number  round_number       name  \
0            0            0             0      Robin   
1            1            1             0    Patrick   
2            2            2             0     Ronnie   
3            3            3             0  Jacquelyn   
4            4            4             0    Phyllis   
..         ...          ...           ...        ...   
93          93           93            13     Ronnie   
94          94           94            13  Jacquelyn   
95          95           95            13    Phyllis   
96          96           96            13      Agnes   
97          97           97            13    Dorothy   

                                   personality_vector  \
0   [-0.25091976  0.90142861  0.46398788  0.197316...   
1   [-0.95883101  0.9398197   0.66488528 -0.575321...   
2   [-0.41754172  0.22370579 -0.72101228 -0.415710...   
3   [ 0.96646177 -0.06647421  0.71988081  0.360615...   
4   [-0.39077246 -0.80465577  0.36846605 -